In [13]:
import numpy as np
import os
import xml.etree.cElementTree as ET
from PIL import Image

#路径用反斜杠，要\\因为一个是转义
#转多txt相关变量
datasrc = "./backdoor/"
datapath = datasrc+"gt.txt"
savepath = datasrc+"txt/"
filepath = datasrc+"file_name.txt"

#转XML相关变量
ANNOTATIONS_DIR_PREFIX = datasrc+"txt/"
DESTINATION_DIR = datasrc+"xml/"

In [14]:
#以下转多txt
def make_dataset(input_path):
    all_imgs = {}
    print("the input path is", input_path)
    with open(input_path,'r') as f:
        print('Parsing annotation files')
        for line in f:
            line_split = line.strip().split(' ')
            print(line_split)
            (filename,useless,x1,y1,x2,y2) = line_split
            if filename not in all_imgs:
                all_imgs[filename] = {}
                
                all_imgs[filename]['filepath'] = 'in'+filename.zfill(6)
                all_imgs[filename]['bboxes'] = []
            #必须是整数吗？
            all_imgs[filename]['bboxes'].append({'class': 'True', 'x1': x1, 'x2': x2, 'y1': y1, 'y2': y2})
            
        all_data = []
        
        for key in all_imgs:
            all_data.append(all_imgs[key])
            
        return all_data
    
def toMutiTXT():
    dataset = make_dataset(datapath)

    for i in range(len(dataset)):
        dataset_path = dataset[i]['filepath']
        dataset_name = dataset_path.split(',')[0]
        sep_dataset = dataset[i]['bboxes']
    
        with open(filepath, 'a') as fn:
            file_name = "{}\n".format(dataset_name)
            fn.write(file_name)
        
        with open(savepath + str(dataset_name)+ ".txt", 'w') as f:
            for j in range(len(sep_dataset)):
                save_dataset = sep_dataset[j]
            
                data = "{} {} {} {} {} \n".format(sep_dataset[j]['class'],sep_dataset[j]['x1'],sep_dataset[j]['y1'], sep_dataset[j]['x2'], sep_dataset[j]['y2'])
            
                f.write(data)
            

In [15]:
# toMutiTXT()

In [16]:
#以下是将多txt转为XML
def create_root(file_prefix, width, height):
    root = ET.Element("annotations")
    ET.SubElement(root, "filename").text = "{}.jpg".format(file_prefix)
    ET.SubElement(root, "folder").text = "images"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = "3"
    return root

def create_object_annotation(root, voc_labels):
    for voc_label in voc_labels:
        obj = ET.SubElement(root, "object")
        ET.SubElement(obj, "name").text = voc_label[0]
        ET.SubElement(obj, "pose").text = "Unspecified"
        ET.SubElement(obj, "truncated").text = str(0)
        ET.SubElement(obj, "difficult").text = str(0)
        bbox = ET.SubElement(obj, "bndbox")
        ET.SubElement(bbox, "xmin").text = str(float(voc_label[1]))
        ET.SubElement(bbox, "ymin").text = str(float(voc_label[2]))
        ET.SubElement(bbox, "xmax").text = str(float(voc_label[3])+float(voc_label[1]))
        ET.SubElement(bbox, "ymax").text = str(float(voc_label[4].split('\\')[0])+float(voc_label[2]))
    return root

def create_file(file_prefix, width, height, voc_labels):
    root = create_root(file_prefix, width, height)
    root = create_object_annotation(root, voc_labels)
    tree = ET.ElementTree(root)
    tree.write(datasrc+"xml/{}.xml".format(file_prefix))

def read_file(file_path):
    file_prefix = file_path.split(".")[0]
    file_path_data = datasrc+"txt/" + file_path
    print("the file path data", file_path_data)
    image_file_name = "{}.jpg".format(file_prefix)
    img = Image.open("{}/{}".format(datasrc+"input", image_file_name))
    w, h = img.size
    with open(file_path_data, 'r') as file:
        lines = file.readlines()
        voc_labels = []
        for line in lines:
            voc = []
            line = line.strip()
            data = line.split()
            print(data[0],data[1],data[2],data[3],data[4])
            voc.append(data[0])
            voc.append(data[1])
            voc.append(data[2])
            voc.append(data[3])
            voc.append(data[4])
            voc_labels.append(voc)
            print(voc_labels)
        create_file(file_prefix, w, h, voc_labels)
    print("Processing complete for file: {}".format(file_path))

def start():
    if not os.path.exists(DESTINATION_DIR):
        os.makedirs(DESTINATION_DIR)
    for filename in os.listdir(ANNOTATIONS_DIR_PREFIX):
#         print(filename)
        if filename.endswith('txt'):
            read_file(filename)
        else:
            print("Skipping file: {}".format(filename))


In [17]:
toMutiTXT()


the input path is ./backdoor/gt.txt
Parsing annotation files
['43', '0', '1', '10', '40', '170']
['44', '0', '1', '11', '43', '170']
['45', '0', '1', '11', '46', '172']
['46', '0', '1', '11', '50', '172']
['47', '0', '1', '10', '52', '172']
['48', '0', '1', '9', '57', '174']
['49', '0', '1', '10', '60', '174']
['50', '0', '4', '10', '60', '174']
['51', '0', '6', '8', '61', '176']
['52', '0', '8', '9', '62', '177']
['53', '0', '12', '8', '63', '176']
['54', '0', '17', '8', '61', '177']
['55', '0', '18', '8', '63', '177']
['56', '0', '21', '7', '63', '179']
['57', '0', '25', '7', '66', '178']
['58', '0', '28', '6', '66', '179']
['59', '0', '32', '7', '64', '178']
['60', '0', '36', '6', '64', '179']
['61', '0', '39', '6', '65', '180']
['62', '0', '43', '6', '65', '180']
['63', '0', '46', '5', '68', '180']
['64', '0', '50', '5', '67', '181']
['65', '0', '53', '5', '68', '181']
['66', '0', '58', '5', '66', '181']
['67', '0', '62', '4', '66', '182']
['68', '0', '66', '4', '66', '182']
['69',

In [18]:
start()


the file path data ./backdoor/txt/in000043.txt
True 1 10 40 170
[['True', '1', '10', '40', '170']]
Processing complete for file: in000043.txt
the file path data ./backdoor/txt/in000044.txt
True 1 11 43 170
[['True', '1', '11', '43', '170']]
Processing complete for file: in000044.txt
the file path data ./backdoor/txt/in000045.txt
True 1 11 46 172
[['True', '1', '11', '46', '172']]
Processing complete for file: in000045.txt
the file path data ./backdoor/txt/in000046.txt
True 1 11 50 172
[['True', '1', '11', '50', '172']]
Processing complete for file: in000046.txt
the file path data ./backdoor/txt/in000047.txt
True 1 10 52 172
[['True', '1', '10', '52', '172']]
Processing complete for file: in000047.txt
the file path data ./backdoor/txt/in000048.txt
True 1 9 57 174
[['True', '1', '9', '57', '174']]
Processing complete for file: in000048.txt
the file path data ./backdoor/txt/in000049.txt
True 1 10 60 174
[['True', '1', '10', '60', '174']]
Processing complete for file: in000049.txt
the file

True 184 2 72 182
[['True', '51', '25', '59', '155'], ['True', '184', '2', '72', '182']]
Processing complete for file: in000101.txt
the file path data ./backdoor/txt/in000102.txt
True 188 2 74 177
[['True', '188', '2', '74', '177']]
True 53 24 59 156
[['True', '188', '2', '74', '177'], ['True', '53', '24', '59', '156']]
Processing complete for file: in000102.txt
the file path data ./backdoor/txt/in000103.txt
True 57 25 58 155
[['True', '57', '25', '58', '155']]
True 191 2 71 175
[['True', '57', '25', '58', '155'], ['True', '191', '2', '71', '175']]
Processing complete for file: in000103.txt
the file path data ./backdoor/txt/in000104.txt
True 193 1 70 177
[['True', '193', '1', '70', '177']]
True 60 25 60 156
[['True', '193', '1', '70', '177'], ['True', '60', '25', '60', '156']]
Processing complete for file: in000104.txt
the file path data ./backdoor/txt/in000105.txt
True 63 24 58 156
[['True', '63', '24', '58', '156']]
True 196 1 71 179
[['True', '63', '24', '58', '156'], ['True', '196'

Processing complete for file: in000153.txt
the file path data ./backdoor/txt/in000154.txt
True 241 1 69 213
[['True', '241', '1', '69', '213']]
True 218 15 23 146
[['True', '241', '1', '69', '213'], ['True', '218', '15', '23', '146']]
Processing complete for file: in000154.txt
the file path data ./backdoor/txt/in000155.txt
True 241 1 70 215
[['True', '241', '1', '70', '215']]
Processing complete for file: in000155.txt
the file path data ./backdoor/txt/in000156.txt
True 241 1 71 217
[['True', '241', '1', '71', '217']]
Processing complete for file: in000156.txt
the file path data ./backdoor/txt/in000157.txt
True 241 1 71 221
[['True', '241', '1', '71', '221']]
Processing complete for file: in000157.txt
the file path data ./backdoor/txt/in000158.txt
True 242 1 71 220
[['True', '242', '1', '71', '220']]
Processing complete for file: in000158.txt
the file path data ./backdoor/txt/in000159.txt
True 242 1 73 225
[['True', '242', '1', '73', '225']]
Processing complete for file: in000159.txt
th

Processing complete for file: in000275.txt
the file path data ./backdoor/txt/in000276.txt
True 48 39 44 137
[['True', '48', '39', '44', '137']]
Processing complete for file: in000276.txt
the file path data ./backdoor/txt/in000277.txt
True 48 39 45 138
[['True', '48', '39', '45', '138']]
Processing complete for file: in000277.txt
the file path data ./backdoor/txt/in000278.txt
True 48 39 46 139
[['True', '48', '39', '46', '139']]
Processing complete for file: in000278.txt
the file path data ./backdoor/txt/in000279.txt
True 49 38 45 138
[['True', '49', '38', '45', '138']]
Processing complete for file: in000279.txt
the file path data ./backdoor/txt/in000280.txt
True 51 38 44 139
[['True', '51', '38', '44', '139']]
Processing complete for file: in000280.txt
the file path data ./backdoor/txt/in000281.txt
True 53 37 40 141
[['True', '53', '37', '40', '141']]
Processing complete for file: in000281.txt
the file path data ./backdoor/txt/in000282.txt
True 57 35 37 142
[['True', '57', '35', '37', 

Processing complete for file: in000334.txt
the file path data ./backdoor/txt/in000335.txt
True 170 31 48 152
[['True', '170', '31', '48', '152']]
Processing complete for file: in000335.txt
the file path data ./backdoor/txt/in000336.txt
True 173 32 52 152
[['True', '173', '32', '52', '152']]
Processing complete for file: in000336.txt
the file path data ./backdoor/txt/in000337.txt
True 175 32 52 151
[['True', '175', '32', '52', '151']]
Processing complete for file: in000337.txt
the file path data ./backdoor/txt/in000338.txt
True 177 33 51 151
[['True', '177', '33', '51', '151']]
Processing complete for file: in000338.txt
the file path data ./backdoor/txt/in000339.txt
True 177 33 52 151
[['True', '177', '33', '52', '151']]
Processing complete for file: in000339.txt
the file path data ./backdoor/txt/in000340.txt
True 178 32 51 152
[['True', '178', '32', '51', '152']]
Processing complete for file: in000340.txt
the file path data ./backdoor/txt/in000341.txt
True 180 32 49 152
[['True', '180'

...

Processing complete for file: in001744.txt
the file path data ./backdoor/txt/in001745.txt
True 207 27 76 164
[['True', '207', '27', '76', '164']]
Processing complete for file: in001745.txt
the file path data ./backdoor/txt/in001746.txt
True 209 27 76 163
[['True', '209', '27', '76', '163']]
Processing complete for file: in001746.txt
the file path data ./backdoor/txt/in001747.txt
True 213 27 72 164
[['True', '213', '27', '72', '164']]
Processing complete for file: in001747.txt
the file path data ./backdoor/txt/in001748.txt
True 214 28 71 162
[['True', '214', '28', '71', '162']]
Processing complete for file: in001748.txt
the file path data ./backdoor/txt/in001749.txt
True 218 27 69 163
[['True', '218', '27', '69', '163']]
Processing complete for file: in001749.txt
the file path data ./backdoor/txt/in001750.txt
True 221 25 67 165
[['True', '221', '25', '67', '165']]
Processing complete for file: in001750.txt
the file path data ./backdoor/txt/in001751.txt
True 226 25 61 165
[['True', '226'

Processing complete for file: in001808.txt
the file path data ./backdoor/txt/in001809.txt
True 231 31 63 171
[['True', '231', '31', '63', '171']]
Processing complete for file: in001809.txt
the file path data ./backdoor/txt/in001810.txt
True 229 30 63 172
[['True', '229', '30', '63', '172']]
Processing complete for file: in001810.txt
the file path data ./backdoor/txt/in001811.txt
True 226 31 65 171
[['True', '226', '31', '65', '171']]
Processing complete for file: in001811.txt
the file path data ./backdoor/txt/in001812.txt
True 224 32 65 170
[['True', '224', '32', '65', '170']]
Processing complete for file: in001812.txt
the file path data ./backdoor/txt/in001813.txt
True 222 32 65 169
[['True', '222', '32', '65', '169']]
Processing complete for file: in001813.txt
the file path data ./backdoor/txt/in001814.txt
True 220 33 65 169
[['True', '220', '33', '65', '169']]
Processing complete for file: in001814.txt
the file path data ./backdoor/txt/in001815.txt
True 219 34 65 167
[['True', '219'

the file path data ./backdoor/txt/in001851.txt
True 41 46 34 130
[['True', '41', '46', '34', '130']]
True 150 39 57 159
[['True', '41', '46', '34', '130'], ['True', '150', '39', '57', '159']]
Processing complete for file: in001851.txt
the file path data ./backdoor/txt/in001852.txt
True 45 45 31 131
[['True', '45', '45', '31', '131']]
True 149 39 57 159
[['True', '45', '45', '31', '131'], ['True', '149', '39', '57', '159']]
Processing complete for file: in001852.txt
the file path data ./backdoor/txt/in001853.txt
True 49 45 27 131
[['True', '49', '45', '27', '131']]
True 148 38 57 159
[['True', '49', '45', '27', '131'], ['True', '148', '38', '57', '159']]
Processing complete for file: in001853.txt
the file path data ./backdoor/txt/in001854.txt
True 51 44 31 131
[['True', '51', '44', '31', '131']]
True 145 38 58 157
[['True', '51', '44', '31', '131'], ['True', '145', '38', '58', '157']]
Processing complete for file: in001854.txt
the file path data ./backdoor/txt/in001855.txt
True 52 45 38

Processing complete for file: in001905.txt
the file path data ./backdoor/txt/in001906.txt
True 153 43 50 138
[['True', '153', '43', '50', '138']]
True 64 41 61 149
[['True', '153', '43', '50', '138'], ['True', '64', '41', '61', '149']]
Processing complete for file: in001906.txt
the file path data ./backdoor/txt/in001907.txt
True 154 40 50 141
[['True', '154', '40', '50', '141']]
True 63 41 61 150
[['True', '154', '40', '50', '141'], ['True', '63', '41', '61', '150']]
Processing complete for file: in001907.txt
the file path data ./backdoor/txt/in001908.txt
True 61 41 60 150
[['True', '61', '41', '60', '150']]
True 158 39 47 142
[['True', '61', '41', '60', '150'], ['True', '158', '39', '47', '142']]
Processing complete for file: in001908.txt
the file path data ./backdoor/txt/in001909.txt
True 162 39 42 142
[['True', '162', '39', '42', '142']]
True 60 41 61 149
[['True', '162', '39', '42', '142'], ['True', '60', '41', '61', '149']]
Processing complete for file: in001909.txt
the file path 

[['True', '265', '38', '48', '149'], ['True', '10', '44', '58', '142']]
Processing complete for file: in001947.txt
the file path data ./backdoor/txt/in001948.txt
True 265 38 52 150
[['True', '265', '38', '52', '150']]
True 10 44 56 141
[['True', '265', '38', '52', '150'], ['True', '10', '44', '56', '141']]
Processing complete for file: in001948.txt
the file path data ./backdoor/txt/in001949.txt
True 265 38 53 150
[['True', '265', '38', '53', '150']]
True 8 45 57 141
[['True', '265', '38', '53', '150'], ['True', '8', '45', '57', '141']]
Processing complete for file: in001949.txt
the file path data ./backdoor/txt/in001950.txt
True 267 37 53 151
[['True', '267', '37', '53', '151']]
True 7 46 57 140
[['True', '267', '37', '53', '151'], ['True', '7', '46', '57', '140']]
Processing complete for file: in001950.txt
the file path data ./backdoor/txt/in001951.txt
True 270 36 51 152
[['True', '270', '36', '51', '152']]
True 5 46 57 139
[['True', '270', '36', '51', '152'], ['True', '5', '46', '57'